# Blood Cell Images for Cancer Detection

image

### Story

# Dependencies

In [1]:
# Standard 
import numpy as np
import pandas as pd

# Visualize
import matplotlib.pyplot as plt
import seaborn as sns

# Images
from torchvision.datasets import ImageFolder
from torchvision.transforms import transforms

# Utility
from torch.utils.data import DataLoader
from sklearn.metrics import confusion_matrix

# Neural network
import torch
import torch.nn as nn
from torch import optim

# Evaluation
from torchmetrics import Precision, Recall

# Transform for images

In [4]:
train_transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.RandomRotation(45),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ToTensor()
])

test_transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor()
])

# Import images

In [7]:
train_data = ImageFolder(
    "data/train",
    transform = train_transform
)

test_data = ImageFolder(
    "data/test",
    transform = test_transform
)

# Create DataLoaders

In [15]:
def CreateDataLoaders(data, batch_size):
    """
    data: 
    Transformed images from train, validation or test folders

    batch_size: 
    Number of images loaded each batch
    """
    x = DataLoader(
        data,
        shuffle = True,
        batch_size = batch_size,
        )
    return x

# DataLoaders
train_dataloader = CreateDataLoaders(train_data, 32)
test_dataloader = CreateDataLoaders(test_data, 32)

# Print images and labels
images, labels = next(iter(train_dataloader))
print(images, labels)

tensor([[[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],

         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],

         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]],


        [[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
        

# Name encoding

In [18]:
names = train_data.class_to_idx
print(names)

{'basophil': 0, 'erythroblast': 1, 'monocyte': 2, 'myeloblast': 3, 'seg_neutrophil': 4}
